In [2]:
# 1. Importer les librairies nécessaires
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd
from google.cloud import storage

import requests
from PIL import Image
import numpy as np
from io import BytesIO

import cv2
import numpy as np
import requests
from io import BytesIO

2024-11-28 17:51:32.136681: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-28 17:51:32.149564: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-28 17:51:32.220196: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-28 17:51:32.305025: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732812692.361674   89897 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732812692.37

In [3]:
# On appelle le dataframe où les noms des fichiers sont stockés
df = pd.read_csv('data_train_set.csv')
df

# Filtrer les noms avec plus de 500 occurrences de file_name
name_counts = df['name'].value_counts()
names_with_more_than_500 = name_counts[name_counts > 1600].index

# Sélectionner les occurrences correspondantes
filtered_df = df[df['name'].isin(names_with_more_than_500)]

# Afficher ou sauvegarder le DataFrame filtré
print(filtered_df)

        file_name                    name               name_readable
39     131216.jpg              tomato-raw                Tomato, raw 
48     131268.jpg                  butter                      Butter
55     131364.jpg  salad-leaf-salad-green  Salad, leaf / salad, green
64     131419.jpg             bread-white                Bread, white
67     131482.jpg              tomato-raw                Tomato, raw 
...           ...                     ...                         ...
70350  130858.jpg              tomato-raw                Tomato, raw 
70361  130896.jpg                  butter                      Butter
70367  130952.jpg  salad-leaf-salad-green  Salad, leaf / salad, green
70386  131015.jpg                   water                       Water
70395  131071.jpg                  butter                      Butter

[10287 rows x 3 columns]


In [5]:
# Associer les chemins des images téléchargées à leurs tags avec un local path
LOCAL_DIRECTORY = "images_sample/"
filtered_df['local_path'] = filtered_df['file_name'].apply(lambda x: os.path.join(LOCAL_DIRECTORY, x))
df_final = filtered_df

/tmp/ipykernel_89897/3532618580.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['local_path'] = filtered_df['file_name'].apply(lambda x: os.path.join(LOCAL_DIRECTORY, x))


In [ ]:
# Transformation en dictionnaire
image_to_name = df_final.set_index('local_path')['name'].to_dict()

# Exemple d'accès à un tag pour une image
for path, name in image_to_name.items():
    print(f"Image: {path}, Name: {name}")

In [ ]:
# Définir le X

import os
import numpy as np
from PIL import Image

# Dimensions souhaitées pour les images
image_size = (128, 128)

# Liste pour stocker les images
images_array = []

# Charger et redimensionner les images
for file_name in os.listdir(LOCAL_DIRECTORY):
    if file_name.endswith('.jpg'):
        file_path = os.path.join(LOCAL_DIRECTORY, file_name)
        try:
            # Ouvrir l'image, redimensionner et convertir en tableau numpy
            img = Image.open(file_path).convert("RGB")
            img = img.resize(image_size)
            img_array = np.array(img)
            images_array.append(img_array)
        except Exception as e:
            print(f"Erreur avec l'image {file_name}: {e}")

# Convertir la liste en un tableau numpy
X = np.array(images_array, dtype="float32") / 255.0  # Normalisation entre 0 et 1

print(f"Tableau des images chargé : {X.shape}")

In [ ]:
# needed to install object_detection library and enlarge labels
! rm -rf ./models && git clone https://github.com/tensorflow/models.git \
    && sed -i "s#ImageFont.truetype('arial.ttf', 24)#ImageFont.truetype('arial.ttf', 50)#g" ./models/research/object_detection/utils/visualization_utils.py \
    && cp /usr/share/fonts/truetype/dejavu/DejaVuSans.ttf /usr/share/fonts/truetype/dejavu/arial.ttf

In [ ]:
# install object_detection library
! cd models/research \
    && protoc object_detection/protos/*.proto --python_out=. \
    && cp object_detection/packages/tf2/setup.py . && \
    python3 -m pip install --use-feature=2020-resolver .

In [ ]:
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import dataset_util, label_map_util
from object_detection.protos import string_int_label_map_pb2

In [ ]:
# reconstruct frozen graph
def reconstruct(pb_path):
    if not os.path.isfile(pb_path):
        print("Error: %s not found" % pb_path)

    print("Reconstructing Tensorflow model")
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(pb_path, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    print("Success!")
    return detection_graph

In [ ]:
# visualize detection
def image2np(image):
    (w, h) = image.size
    return np.array(image.getdata()).reshape((h, w, 3)).astype(np.uint8)

def image2tensor(image):
    npim = image2np(image)
    return np.expand_dims(npim, axis=0)

%matplotlib inline
def detect(detection_graph, test_image_path):
    with detection_graph.as_default():
        gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.01)
        with tf.compat.v1.Session(graph=detection_graph,config=tf.compat.v1.ConfigProto(gpu_options=gpu_options)) as sess:
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            image = Image.open(test_image_path)
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image2tensor(image)}
            )

            npim = image2np(image)
            vis_util.visualize_boxes_and_labels_on_image_array(
                npim,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=15)
            plt.figure(figsize=(12, 8))
            plt.imshow(npim)
            plt.show()